# 라이브러리 설치

In [1]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch
import json
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 데이터와 모델로드

In [3]:

# 🧾 1. 데이터 불러오기 (JSON → HuggingFace Dataset)
with open("dataset_3labels_class_train.json", "r", encoding="utf-8") as f:
    data = json.load(f)

dataset = Dataset.from_list(data)
dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_ds, eval_ds = dataset["train"], dataset["test"]

# 🔤 2. Tokenizer 및 Tokenizing 함수
model_name = "klue/bert-base"
tokenizer = BertTokenizer.from_pretrained(model_name)

def tokenize(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=128)

train_ds = train_ds.map(tokenize, batched=True)
eval_ds = eval_ds.map(tokenize, batched=True)

# ❌ 원본 텍스트 제거, ✅ 라벨을 정수로 지정
train_ds = train_ds.remove_columns(["text"]).with_format("torch")
eval_ds = eval_ds.remove_columns(["text"]).with_format("torch")

# 🧠 3. 모델 불러오기
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)

# 🎯 4. Metrics
def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds)
    }


Map: 100%|██████████| 15/15 [00:00<00:00, 3044.94 examples/s]
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# 학습(hpara 설정해보기, 결과 모델 경로 지정)

In [4]:

# 5. Trainer 설정
training_args = TrainingArguments(
    output_dir="./results",
    # evaluation_strategy="epoch",
    save_strategy="no",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    seed=42
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# 🚀 6. 학습 시작
trainer.train()
trainer.save_model("./results")  # 직접 저장 시도

/opt/conda/lib/python3.11/site-packages/torch/cuda/__init__.py:789: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/tmp/ipykernel_397089/1643025717.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/opt/conda/lib/python3.11/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,0.858200
20,0.497600


# 테스트

In [ ]:
import json
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# 1. JSON 파일 읽기
with open("dataset_3labels_class_test.json", "r", encoding="utf-8") as f:
    data = json.load(f)
    # data 예: [{"text": "문장1", "label": 0}, {"text": "문장2", "label": 1}, ...]

# 2. 모델, 토크나이저 불러오기
# model_name = "klue/bert-base"
# tokenizer = BertTokenizer.from_pretrained(model_name)
# model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)
model_dir = "./results"
model = BertForSequenceClassification.from_pretrained(model_dir)
tokenizer = BertTokenizer.from_pretrained(model_dir)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

label_map = {0: "HR 관련 문의", 1: "업무 분장 관련", 2: "HR, 업무 분장 양방향으로 해석 가능"}

results = []

# 3. 데이터마다 추론
with torch.no_grad():
    for item in data:
        text = item["text"]
        true_label = item["label"]

        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        outputs = model(**inputs)
        logits = outputs.logits

        probs = torch.softmax(logits, dim=1)
        confidence, predicted_class = torch.max(probs, dim=1)
        predicted_class = predicted_class.item()
        confidence = confidence.item()

        is_correct = "O" if predicted_class == true_label else "X"

        results.append((
            label_map[predicted_class],  # 예측 레이블명
            label_map[true_label],       # 실제 레이블명
            is_correct,
            confidence,
            text
        ))

# 4. confidence 기준 내림차순 정렬
results = sorted(results, key=lambda x: x[3], reverse=True)

correct = sum(1 for r in results if r[2] == "O")
total = len(results)
accuracy = correct / total * 100

# 5. 결과 출력
print(f"{'예측':<10} {'정답':<10} {'정오':<2} {'Confidence':<10} 문장")
print("-" * 70)
for pred_label, true_label, is_correct, score, text in results:
    print(f"[{pred_label:<8}] [{true_label:<8}] {is_correct}  ({score:.4f})  {text}")

print(f"\n✅ 최종 정확도: {correct}/{total} ({accuracy:.2f}%)")


예측         정답         정오 Confidence 문장
----------------------------------------------------------------------
[업무 분장 관련] [업무 분장 관련] O  (0.9996)  법률 자문 부서 누구야?
[업무 분장 관련] [업무 분장 관련] O  (0.9996)  법률 자문 부서 누구야?
[업무 분장 관련] [업무 분장 관련] O  (0.9995)  회의실 관리팀 누구야?
[업무 분장 관련] [업무 분장 관련] O  (0.9995)  복지포인트 담당 부서 누구야?
[업무 분장 관련] [업무 분장 관련] O  (0.9995)  복지포인트 담당 부서 누구야?
[업무 분장 관련] [업무 분장 관련] O  (0.9995)  복지포인트 담당 부서 누구야?
[업무 분장 관련] [업무 분장 관련] O  (0.9995)  점심 식사 담당 부서 누구야?
[업무 분장 관련] [업무 분장 관련] O  (0.9995)  점심 식사 담당 부서 누구야?
[업무 분장 관련] [업무 분장 관련] O  (0.9995)  외부 방문자 안내 담당자 누구야?
[업무 분장 관련] [업무 분장 관련] O  (0.9995)  긴급 상황 담당자 누구야?
[업무 분장 관련] [업무 분장 관련] O  (0.9995)  근태 관리 부서 누구야?
[업무 분장 관련] [업무 분장 관련] O  (0.9995)  근태 관리 부서 누구야?
[업무 분장 관련] [업무 분장 관련] O  (0.9995)  법인카드 담당자 누구야?
[HR 관련 문의] [HR 관련 문의] O  (0.9995)  증명서 발급 방법 어떻게 해?
[업무 분장 관련] [업무 분장 관련] O  (0.9995)  IT 지원팀 누구야?
[업무 분장 관련] [업무 분장 관련] O  (0.9995)  IT 지원팀 누구야?
[업무 분장 관련] [업무 분장 관련] O  (0.9995)  IT 지원팀 누구야?
[HR 관련 문의] [HR 관련 문의] O  (0.9995)  명함 제작